In [1]:
import warnings
warnings.filterwarnings('ignore')

# all import statements
import numpy as np
import pandas as pd
import pydicom as pyd
import os
import matplotlib.pyplot as plt
import mudicom

from os.path import dirname, join
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from PIL import Image

from scipy.misc import imresize
from scipy.signal import convolve2d
import scipy
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

from numpy import newaxis

from skimage.measure import shannon_entropy
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy


import math
from skimage.measure import label, regionprops

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import matplotlib.patches as mpatches

import statistics

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

import cv2
from skimage import img_as_float
from skimage import exposure

In [2]:
import pickle

from tqdm import tqdm
import os
import sys

In [3]:
def load_pickle(fileName):
    with open(fileName, "rb") as fp:
        file = pickle.load(fp)
    return file 

def write_pickle(file, fileName):
    with open(fileName, "wb") as fp:
        pickle.dump(file, fp)

### GLCM

In [4]:

def s_entropy(dcm):
    return shannon_entropy(dcm)

def entropy_simple(dcm):
    return entropy(dcm)

class glcm:
    def __init__(self, image):
        distance = [1, 2, 3]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        self.image = img_as_ubyte(image.astype('int64'))
        self.glcm_mat = greycomatrix(self.image, distances = distance, angles = angles, symmetric = True, normed = True)
        self.properties = ['correlation', 'homogeneity', 'contrast', 'energy']
            
    def correlation(self):
        return greycoprops(self.glcm_mat, 'correlation').flatten()
    
    def homogeneity(self):
        return greycoprops(self.glcm_mat, 'homogeneity').flatten()
    
    def contrast(self):
        return greycoprops(self.glcm_mat, 'contrast').flatten()
    
    def energy(self):
        return greycoprops(self.glcm_mat, 'energy').flatten()
    
    def glcm_all(self):
        return np.hstack([greycoprops(self.glcm_mat, props).ravel() for props in self.properties])
    

### Region Props

In [5]:
# thresh_ada = threshold_otsu(image_adahist)
# bw_ada = closing(image_adahist > thresh_ada, square(3))
# bw_clear_ada = clear_border(bw_ada)

class region_props:
    def __init__(self, image):
        self.image = image
        self.thresh = threshold_otsu(self.image)
        self.bw = closing(self.image > self.thresh, square(3))
        self.bw_clear = clear_border(self.bw)
        self.bw_label = label(self.bw_clear)
        self.regions = regionprops(self.bw_label)
#         self.idx = 0
        
    def plot_image_bw(self):
        plt.imshow(self.bw, cmap = plt.cm.bone)
    
    def plot_image(self):
        plt.imshow(self.bw_clear, cmap=plt.cm.bone)
        
    def plot_image_with_labels(self):
        plt.imshow(self.bw_label, cmap=plt.cm.bone)
        
    def max_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return max(lista)
    
    def eccentricity(self):
        
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].eccentricity
    
    def euler_number(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].euler_number
    
    def solidity(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].solidity
    
    def perimeter(self):
        lista = []
        for e in self.regions:
            lista.append(e.perimeter)
        return max(lista)
    
    def mean_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        
        return statistics.mean(lista)
        
    
    def std_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>2:
            return statistics.stdev(lista)
        else:
            return -1
    
    def thresh_img(self):
        return self.thresh
        

### Moments

In [6]:
class moments:
    
    def __init__(self, image):
        self.image = image
        self.moment = cv2.moments(self.image)
        self.hu = cv2.HuMoments(self.moment)
        
    def get_moments(self):
#         keys = [key for key in self.moment.keys()]
        values = [value for value in self.moment.values()]
        return values
    
    def get_HuMoments(self):
        moments_hu = []
        for m in range(len(self.hu)):
            moments_hu.append(self.hu[m][0])
        return moments_hu

#### DataFrame dcm_1

In [7]:
dcm_2 = load_pickle("dcm-2")
pat_2 = load_pickle("pat-2")

In [8]:
len(dcm_2), len(pat_2)

(4000, 4000)

In [9]:
from tqdm import tqdm
import os
import sys

In [10]:
data_dcm_2_ = dict()
for dc in tqdm(range(len(dcm_2))):
    data_dcm_2_[pat_2[dc]] = [s_entropy(dcm_2[dc]), entropy_simple(dcm_2[dc]), moments(dcm_2[dc]).get_moments(), moments(dcm_2[dc]).get_HuMoments(),
                      region_props(dcm_2[dc]).max_area(), region_props(dcm_2[dc]).eccentricity(), region_props(dcm_2[dc]).euler_number(),
                      region_props(dcm_2[dc]).solidity(), region_props(dcm_2[dc]).perimeter(), region_props(dcm_2[dc]).mean_area(), 
                      region_props(dcm_2[dc]).std_area(), region_props(dcm_2[dc]).thresh_img(), glcm(dcm_2[dc]).correlation(), glcm(dcm_2[dc]).homogeneity(),
                      glcm(dcm_2[dc]).contrast(), glcm(dcm_2[dc]).energy()]


100%|██████████| 4000/4000 [54:27<00:00,  1.24it/s]


In [11]:
df_dcm_2 = pd.DataFrame(data_dcm_2_).T

In [12]:
df_dcm_2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
4dcbea60-aaeb-4656-a64d-278d82a787e1,7.18481,4.98013,"[170073996.0, 86663614151.0, 88750054199.0, 58...","[0.001009765391671003, 5.779607804931761e-10, ...",241,0.979613,1,0.625974,103.977,10,27.2947,157,"[0.9976148704867444, 0.996647739346442, 0.9982...","[0.4941947535016135, 0.43521053392522513, 0.56...","[8.166931092680835, 11.46305262443754, 5.90906...","[0.03866420332052904, 0.03509654057638883, 0.0..."
fa3038f0-67dd-43dd-89b2-a926afe329d9,7.57574,5.2511,"[118280122.0, 61834875029.0, 74358558428.0, 41...","[0.001285104073331831, 6.503068314377632e-10, ...",1086,0.829854,-1,0.479047,363.765,36,115.806,110,"[0.99553374511403, 0.989516096561175, 0.993313...","[0.5694614124304528, 0.5011006661021367, 0.597...","[32.21750901148581, 75.48069666487977, 48.2085...","[0.06292689628630174, 0.06022050091016017, 0.0..."
5d605f45-8f8b-4f75-8abc-60a004c2b419,6.91534,4.79335,"[159761246.0, 87974122127.0, 87825470614.0, 60...","[0.0009638591159286361, 1.473536379949066e-09,...",734,0.582896,1,0.886473,139.556,262,360.137,99,"[0.9976546797702236, 0.9950137436594705, 0.997...","[0.64543073734383, 0.5657755323545853, 0.66421...","[15.64593356702101, 33.195742306233264, 17.778...","[0.06139752461480512, 0.055080661186017746, 0...."
0c0181c0-c009-4387-b087-d314a4ffcba2,7.59477,5.26429,"[165045416.0, 84555926198.0, 95711006975.0, 57...","[0.001033303924492152, 4.675710900220722e-10, ...",241,0.5786,1,0.126976,233.278,10,31.0483,142,"[0.9927304283621905, 0.9912663701713202, 0.998...","[0.5057900928104287, 0.435293582067239, 0.5321...","[46.52882911779076, 55.79319541073396, 11.7466...","[0.03418050659752694, 0.03047161278934389, 0.0..."
83691ce2-bb15-42b0-a2bc-6383bd3c3a29,7.37317,5.11069,"[175634104.0, 88823973259.0, 98185017791.0, 59...","[0.0009612281745803628, 7.30226580716377e-10, ...",550,0.835751,1,0.946644,92.4264,19,73.1369,150,"[0.9979890112749176, 0.9953770305304854, 0.997...","[0.5823956412331177, 0.4886409748406707, 0.574...","[11.802393580461878, 27.069941683412495, 16.00...","[0.04771779748607389, 0.041313599716824356, 0...."


In [13]:
df_dcm_2.to_pickle('df_dcm_2')

In [14]:
df_dcm_2.to_csv('df_dcm_2.csv')